# Channel Flow - Benchmark Setting ReTau = 395 

Results published: preliminary study 


### Preliminaries

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 4
Count: 1



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 170
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in D:\BoSSS-experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 97
   at Submission#67.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## Initialization tasks

Loading the `KOmegaSolver` and additional namespaces:

In [ ]:
using BoSSS.Application.KOmega;
using BoSSS.Application.RANSCommon;
using BoSSS.Solution.NSECommon;
using NUnit.Framework;

Initialization of the Workflow management; there is the project name `ChannelFlowKOmega` which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("ChannelFlowKOmega");

Project name is set to 'ChannelFlowKOmega'.
Opening existing database 'D:\local\ChannelFlowKOmega'.


In [ ]:
OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\smuda\ChannelFlowKOmega");

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\ChannelFlowKOmega'.


In [ ]:
databases

#0: { Session Count = 42; Grid Count = 83; Path = \\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D }
#1: { Session Count = 3; Grid Count = 4; Path = D:\local\ChannelFlowKOmega }
#2: { Session Count = 3; Grid Count = 1; Path = \\hpccluster\hpccluster-scratch\smuda\ChannelFlowKOmega }


Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ],,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,,[ \\hpccluster\hpccluster-scratch\smuda\ == ],FDY\smuda,DC2,<null>,Normal,True


In [ ]:
var myBatch = ExecutionQueues[1];
//var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,FDY\smuda,DC2,<null>,Normal,True,[ \\hpccluster\hpccluster-scratch\smuda\ == ]


In [ ]:
wmg.Sessions

#0: ChannelFlowKOmega	CFKO_J162k1	04/06/2022 10:40:48	b28c5363...
#1: ChannelFlowKOmega	CFKO_J722k1	03/30/2022 13:44:32	d8522cc2...
#2: ChannelFlowKOmega	CFKO_J741k1	02/25/2022 14:08:43	908240ab...
#3: ChannelFlowKOmega	CFKO_J162k3_p3_HomotopyApproach2*	05/17/2022 18:39:59	0c6ae1bb...
#4: ChannelFlowKOmega	CFKO_J162k3_p3_HomotopyApproach1	05/17/2022 17:06:18	dc9452ba...
#5: ChannelFlowKOmega	CFKO_J162k3_p3*	05/17/2022 16:02:51	8098acec...


## Grid Creation - Channel

In [ ]:
// channel length
double l = 0.2;
// channel half-width
double delta = 1;

(int cellsx, int cellsy)[] Resolutions = new [] { (10, 10) };
IGridInfo[] Grids = new IGridInfo[Resolutions.Length];

bool periodicBC = true;

In [ ]:
double[] _xNodes;
double[] _yNodes;
for(int i = 0; i < Resolutions.Length; i++) {
    var Res = Resolutions[i];
    string GridName = $"ChannelFlowKOmega_{Res.cellsx}x{2*Res.cellsy}_TanhSpacing";

    // grid metrics
    _xNodes = GenericBlas.Linspace(0, l, Res.cellsx);
    var _yNodesTop = Grid1D.TanhSpacing(delta, 2.0*delta, Res.cellsy, 1.8, false);
    _yNodesTop = _yNodesTop.GetSubVector(1, (_yNodesTop.Length - 1));
    var _yNodesBottom = Grid1D.TanhSpacing(0, delta, Res.cellsy, 1.8, true);  
    _yNodes = ArrayTools.Cat(_yNodesBottom, _yNodesTop);


    IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
    cachedGrid = null;
    if(cachedGrid == null) {
        
        // must create new Grid
        var grd = Grid2D.Cartesian2DGrid(_xNodes,
                                         _yNodes,
                                         CellType.Square_Linear,
                                         periodicX: periodicBC
                                         );
        grd.Name = GridName;

        grd.DefineEdgeTags(delegate(Vector X) {
            string ret = null;

            if (Math.Abs(X.y - (0)) < 1.0e-8)
                // bottom
                ret = IncompressibleBcType.Wall.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString();

            if (Math.Abs(X.y - (2.0*delta)) < 1.0e-8)
                // top
                ret = IncompressibleBcType.Wall.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString();

            if (!periodicBC) {
                if (Math.Abs(X.x - (0)) < 1.0e-8)
                    // left
                    ret = IncompressibleBcType.Velocity_Inlet.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString();

                if (Math.Abs(X.x - (l)) < 1.0e-8)
                    // right
                    ret = IncompressibleBcType.Pressure_Outlet.ToString() + "_" + TurbulenceModelBcType.Neumann.ToString();
            }

            return ret;
        });     
        
        Grids[i] = wmg.SaveGrid(grd);
        
    } else {
        //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
        Console.WriteLine("Grid already found in database - identifid by name " + GridName);
        Grids[i] = cachedGrid;
    }
    
}

Grid Edge Tags changed.
An equivalent grid (2274c87d-c23b-485f-a1ce-2308d1024693) is already present in the database -- the grid will not be saved.


In [ ]:
//databases.Pick(2).Grids

In [ ]:
double cellRatio = _yNodes[1] / _xNodes[1];
cellRatio

1.1947385361340517

## Initial Values

In [ ]:
var VelXInit = new Formula(
    "VelX",
    false,
    "using ilPSP.Utils; " + 
    "double VelX(double[] X) { " + 
    "    double U0 = 197.5;" + 
    "    double delta = 1.0;" + 
    "    return U0 * (X[1]/delta) * (2 - (X[1]/delta)); " + 
    "}");

In [ ]:
var kPolyInit = new Formula(
    "kPoly",
    false,
    "using ilPSP.Utils; " + 
    "double kPoly(double[] X) { " + 
    "    double U0 = 197.5;" + 
    "    double k0 = 0.5 * U0 * U0;" + 
    "    double delta = 1.0;" + 
    "    return k0 * (X[1]/delta) * (2 - (X[1]/delta)); " + 
    "}");

In [ ]:
var omegaTildeInit = new Formula(
    "omegaTilde",
    false,
    "using ilPSP.Utils; " + 
    "double omegaTilde(double[] X) { " + 
    "    double turbulentIntensity = 0.8;" + 
    "    double Umean= 1.0;" + 
    "    double kIn = 3 / 2.0 * Math.Pow(Umean * turbulentIntensity, 2);" + 
    "    double L = 0.4;" +
    "    double omegaTildeIn = Math.Log(Math.Sqrt(kIn) / L);" + 
    "    double omegaTildeWall = Math.Log(6e2);" + 
    "    double height = 1.0;" + 
    "    return ((omegaTildeIn - omegaTildeWall) * (1 - (X[1] / height) * (X[1] / height))) + omegaTildeWall; " + 
    "}");

## Setup of control objects for all solver runs

In [ ]:
double dyWall = _yNodes[1];
double u_tau = 1.0;
double viscosity = 0.002531645569620;
double dyWallPlus = u_tau * dyWall / viscosity;
dyWallPlus

10.487149372733281

In [ ]:
int pOrder = 3;
double dyWallPlusDG = dyWallPlus / ((double)pOrder + 1.0); 
dyWallPlusDG

2.6217873431833203

In [ ]:
double tau_w = 1.0;
double beta = 3.0/40.0;
double density = 1.0;
double[] alpha_pS = new double[] {0.37, 8.21e-2, 3.57e-2, 1.99e-2, 1.27e-2}; // for orders p = {0, 1, 2, 3, 4}
double alpha_p = alpha_pS[pOrder];
double omegaT_w = Math.Log(6*tau_w/(beta*density*viscosity)) + (2.0*Math.Log(1.0/alpha_p)) + (2*Math.Log(1.0/dyWallPlusDG));
omegaT_w

16.267270939597523

In [ ]:
List<KOmegaControl> Controls = new List<KOmegaControl>();
Controls.Clear();

int[] DegreeS = new int[] { pOrder };

foreach(int k in DegreeS) {
foreach(var grd in Grids.Take(1)) {

    long J = grd.NumberOfCells;
    string JobName = $"CFKO_J{J}k{k}_p{pOrder}_decoupled";
    Console.WriteLine("Case: " + JobName);

    var C = new KOmegaControl();
    
    C.SetGrid(grd);
    C.SetDGdegree(k);
    C.SessionName = JobName;
    

    C.NonLinearSolver.MaxSolverIterations = 100;
    C.NonLinearSolver.MinSolverIterations = 3;
    C.NonLinearSolver.ConvergenceCriterion = 1E-8;
    // C.LinearSolver.MaxSolverIterations = 10;
    // C.LinearSolver.MinSolverIterations = 3;
    // C.LinearSolver.ConvergenceCriterion = 1E-10;
    C.dt = 1e20; // pseudo-timestep
    // C.dt = 100;
    C.dtMax = C.dt;
    C.dtMin = C.dt;
    // C.dtMax = 0.001;
    // C.dtMin = 100.0;
    C.Timestepper_Scheme = KOmegaControl.TimesteppingScheme.ImplicitEuler;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    // C.LinearSolver.SolverCode = LinearSolverCode.classic_mumps;
    // C.LinearSolver.SolverCode = LinearSolverCode.exp_gmres_levelpmg;
    // C.LinearSolver.SolverCode = LinearSolverCode.exp_direct_lapack;
    C.LinearSolver = LinearSolverCode.classic_mumps.GetConfig();
    // C.LinearSolver.SolverCode = LinearSolverCode.exp_softgmres;

    //C.NoOfTimesteps = 100;

    C.AdaptiveMeshRefinement = false;
    C.RefinementLevel = 10;

    C.UseArtificialDiffusion = false;

    C.Bodyforces = false;
    //C.WhichWall = "Wall_Cylinder";

    //Debugging and Solver Analysis
    C.OperatorMatrixAnalysis = false;
    C.AnalysisLevel = 2;
    C.SetInitialConditions = true;
    C.SetInitialPressure = false;
    C.SetParamsAnalyticalSol = false;
    C.ComputeL2Error = false;
    C.GravitySource = false;
    C.GravityX = (X, t) => 0;
    C.GravityY = (X, t) => 0;

    //Physical Params
    C.Stokes = false;
    C.FixedStreamwisePeriodicBC = periodicBC;
    C.Viscosity = viscosity;

    C.homotopy_komega_fadein = 0.0;
    //C.HomotopyApproach = KOmegaControl.HomotopyType.Manual;

    if (C.HomotopyApproach == KOmegaControl.HomotopyType.Manual){
        C.AlphaStarHomotopyValues    = new double[] { 0,0.001,             0.1,             1.0             };
        C.AlphaHomotopyValues        = new double[] { 0,0.001 * C.alpha,    0.1 * C.alpha,    1.0 * C.alpha    };
        C.BetaStarHomotopyValues     = new double[] { 0,0.001 * C.beta_star,0.1 * C.beta_star,1.0 * C.beta_star};
        C.BetaHomotopyValues         = new double[] { 0,0.001 * C.beta,     0.1 * C.beta,     1.0 * C.beta     };
        C.SigmaStarHomotopyValues    = new double[] { 0,1 * C.sigma_star,   1 * C.sigma_star, 1 * C.sigma_star };
        C.SigmaHomotopyValues        = new double[] { 0,1 * C.sigma,        1 * C.sigma,      1 * C.sigma      };
        C.KOmegaFadeinHomotopyValues = new double[] { 0,1,                 1,               1               };
    }

    C.useJacobianForOperatorMatrix = true;

    //Penalties
    C.ViscousPenaltyScaling = 1;
    C.Penalty2 = 1;
    C.Penalty1[0] = 0.0;
    C.Penalty1[1] = 0.0;
    C.PresPenalty2 = 1;
    C.PresPenalty1[0] = 0.0;
    C.PresPenalty1[1] = 0.0;

    // Set Initial Conditions
    C.InitialValues.Add(VariableNames.Velocity_d(0), VelXInit);
    C.InitialValues.Add(VariableNames.Velocity_d(1), new Formula("(X) => 0.0", false));

    C.InitialValues.Add("k", kPolyInit);
    C.InitialValues.Add("omega_tilde", new Formula("(X) => 16.267270939597523", false));

    // Set Boundary Conditions
    C.AddBoundaryValue(IncompressibleBcType.Wall.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), "VelocityX", new Formula("(X) => 0.0", false));
    C.AddBoundaryValue(IncompressibleBcType.Wall.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), "VelocityY", new Formula("(X) => 0.0", false));
    C.AddBoundaryValue(IncompressibleBcType.Wall.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), "k", new Formula("(X) => 0.0", false)); // kWall
    C.AddBoundaryValue(IncompressibleBcType.Wall.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), "omega_tilde", new Formula("(X) => 16.267270939597523", false)); // omegaWall

    if (!C.FixedStreamwisePeriodicBC) {
        C.AddBoundaryValue(IncompressibleBcType.Velocity_Inlet.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), VariableNames.Velocity_d(0), VelXInit);
        C.AddBoundaryValue(IncompressibleBcType.Velocity_Inlet.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), VariableNames.Velocity_d(1), new Formula("(X) => 0.0", false));
        C.AddBoundaryValue(IncompressibleBcType.Velocity_Inlet.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), "k", kPolyInit); // kIn // kWall
                         
        C.AddBoundaryValue(IncompressibleBcType.Velocity_Inlet.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), "omega_tilde",  omegaTildeInit); // omegaIn // omegaWall
                          
        // //C.AddBoundaryCondition("Velocity_inlet",VariableNames.Pressure, Pressurefunction);
        // C.AddBoundaryValue("Pressure_Outlet_neumann");
        C.AddBoundaryValue(IncompressibleBcType.Velocity_Inlet.ToString() + "_" + TurbulenceModelBcType.Dirichlet.ToString(), VariableNames.Pressure, new Formula("(X) => 1.0", false));
        C.AddBoundaryValue(IncompressibleBcType.Pressure_Outlet.ToString() + "_" + TurbulenceModelBcType.Neumann.ToString(), VariableNames.Pressure, new Formula("(X) => 0.0", false));
        // C.AddBoundaryValue("Pressure_Outlet_neumann", "k", X =>
        //                    kPolynomial (X,0)
        //                    // kIn
        //                    // kWall
        //                    );
        // C.AddBoundaryValue("Pressure_Outlet_neumann", "omega", X =>
        //                    omegaPolynomial (X,0)
        //                    // omegaIn
        //                    // omegaWall
        //                    );
        C.AddBoundaryValue(IncompressibleBcType.Pressure_Outlet.ToString() + "_" + TurbulenceModelBcType.Neumann.ToString());

        // C.AddBoundaryValue("Wall_top");
        // C.AddBoundaryValue("Wall_bottom");
        // C.AddBoundaryValue("Velocity_inlet");
        // C.AddBoundaryValue("Pressure_Outlet_neumann");
    }
    
    
    C.TracingNamespaces = "*";
    
    Controls.Add(C);
    
}
}

Case: CFKO_J162k3_p3_decoupled


In [ ]:
int NC = Controls.Count;
for(int i = 0; i < NC; i++) {
    for(int j = 0; j < NC; j++) {
        if(i == j)
            Assert.IsTrue(Controls[i].Equals(Controls[j]), "Control is not self-equal");
        else
            Assert.IsFalse(Controls[i].Equals(Controls[j]), "Different Control are wrongly equal");
    }
}

## Launch Jobs

In [ ]:
Controls.ElementAt(0).SetDatabase(databases.Pick(2));

In [ ]:
Controls.ElementAt(0).GetDatabase()

{ Session Count = 3; Grid Count = 1; Path = \\hpccluster\hpccluster-scratch\smuda\ChannelFlowKOmega }

In [ ]:
foreach(var ctrl in Controls) {
    var oneJob              = ctrl.CreateJob();
    oneJob.NumberOfMPIProcs = 1;
    oneJob.Activate(myBatch); 
}

Deploying job CFKO_J162k3_p3_decoupled ... 
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\smuda\binaries\ChannelFlowKOmega-KOmegaSolver2022May18_134854
copied 46 files.
   written file: control.obj
deployment finished.



In [ ]:
wmg.AllJobs

Loading session 4c57ccf5-bbef-4526-bcf5-82c8996993f6 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\ChannelFlowKOmega\sessions\4c57ccf5-bbef-4526-bcf5-82c8996993f6\Session.info'.'
Loading session 4c57ccf5-bbef-4526-bcf5-82c8996993f6 failed with message 'Could not find file '\\hpccluster\hpccluster-scratch\smuda\ChannelFlowKOmega\sessions\4c57ccf5-bbef-4526-bcf5-82c8996993f6\Session.info'.'


#0: CFKO_J162k3_p3_HomotopyApproach1: FinishedSuccessful (MS HPC client  FDY-WindowsHPC @DC2, @\\hpccluster\hpccluster-scratch\smuda\binaries)	CFKO_J162k3_p3_HomotopyApproach1: FinishedSuccessful (MS HPC client  FDY-WindowsHPC @DC2, @\\hpccluster\hpccluster-scratch\smuda\binaries)
#1: CFKO_J162k3_p3_HomotopyApproach2: InProgress (MS HPC client  FDY-WindowsHPC @DC2, @\\hpccluster\hpccluster-scratch\smuda\binaries)	CFKO_J162k3_p3_HomotopyApproach2: InProgress (MS HPC client  FDY-WindowsHPC @DC2, @\\hpccluster\hpccluster-scratch\smuda\binaries)
#2: CFKO_J162k3_p3_decoupled: InProgress (MS HPC client  FDY-WindowsHPC @DC2, @\\hpccluster\hpccluster-scratch\smuda\binaries)	CFKO_J162k3_p3_decoupled: InProgress (MS HPC client  FDY-WindowsHPC @DC2, @\\hpccluster\hpccluster-scratch\smuda\binaries)


In [ ]:
// wait for all jobs to finish (up to 5 days, check every 30 minutes)
//BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600), PollingIntervallSeconds:(60*30));

In [ ]:
// detect failed Jobs in the job management
/*
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects
*/

In [ ]:
//suspects.Count()

In [ ]:
//NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

### Inspect the output of some arbitrary job:

In [ ]:
//BoSSSshell.WorkflowMgm.AllJobs.First().Value.ShowOutput();

In [ ]:
//wmg.Sessions[0].Export().WithSupersampling(2).Do()

In [ ]:
//wmg.Sessions.Pick(0).Timesteps